# Debug Form Detection Issues

This notebook investigates why example forms are not being detected as forms when compared against themselves.

In [1]:
# Setup
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
from transformers import CLIPProcessor, CLIPModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
import fitz  # PyMuPDF
import pandas as pd

In [3]:
# Load CLIP model
model_name = "openai/clip-vit-large-patch14-336"
clip_model = CLIPModel.from_pretrained(model_name)
clip_processor = CLIPProcessor.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
clip_model = clip_model.to(device)
clip_model.eval()

print(f"Model loaded on {device}")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Model loaded on mps


In [4]:
# Test 1: Extract features from a single example form
example_forms_dir = "../../data/raw/_exampleforms/"
example_files = [f for f in os.listdir(example_forms_dir) if f.endswith('.pdf')]
test_file = example_files[0]

print(f"Testing with: {test_file}")

# Load the test image
pdf_path = os.path.join(example_forms_dir, test_file)
pdf = fitz.open(pdf_path)
page = pdf[0]
pix = page.get_pixmap()
img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
pdf.close()

# Extract features
inputs = clip_processor(images=img, return_tensors="pt").to(device)
with torch.no_grad():
    features1 = clip_model.get_image_features(**inputs)
    features1 = features1.cpu().numpy()
    
print(f"Features shape: {features1.shape}")
print(f"Features norm before normalization: {np.linalg.norm(features1)}")

# Normalize
features1_norm = features1 / np.linalg.norm(features1, axis=1, keepdims=True)
print(f"Features norm after normalization: {np.linalg.norm(features1_norm)}")

Testing with: 25581-000.pdf
Features shape: (1, 768)
Features norm before normalization: 16.98063087463379
Features norm after normalization: 0.9999999403953552


In [7]:
# Test 2: Compare the same image with itself
# Load the same image again
pdf = fitz.open(pdf_path)
page = pdf[0]
pix = page.get_pixmap()
img2 = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
pdf.close()

# Extract features again
inputs2 = clip_processor(images=img2, return_tensors="pt").to(device)
with torch.no_grad():
    features2 = clip_model.get_image_features(**inputs2)
    features2 = features2.cpu().numpy()
    features2_norm = features2 / np.linalg.norm(features2, axis=1, keepdims=True)

# Compare
similarity = cosine_similarity(features1_norm, features2_norm)[0][0]
print(f"\nSimilarity of the same image with itself: {similarity:.6f}")
print(f"This should be 1.0 or very close to 1.0")


Similarity of the same image with itself: 1.000000
This should be 1.0 or very close to 1.0


In [9]:
# Test 3: Load all example features as done in the main notebook
example_features = []

for example_file in example_files[:5]:  # Test with first 5
    example_path = os.path.join(example_forms_dir, example_file)
    
    try:
        pdf = fitz.open(example_path)
        page = pdf[0]
        pix = page.get_pixmap()
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        pdf.close()
        
        # Extract features
        inputs = clip_processor(images=img, return_tensors="pt").to(device)
        with torch.no_grad():
            features = clip_model.get_image_features(**inputs)
            features = features.cpu().numpy()
            features = features / np.linalg.norm(features, axis=1, keepdims=True)
            example_features.append(features)
        
        print(f"Loaded {example_file}: shape {features.shape}")
        
    except Exception as e:
        print(f"Error loading {example_file}: {e}")

print(f"\nTotal example features loaded: {len(example_features)}")

Loaded 25581-000.pdf: shape (1, 768)
Loaded 99171-000.pdf: shape (1, 768)
Loaded 13924-002.pdf: shape (1, 768)
Loaded 1197-000.pdf: shape (1, 768)
Loaded 67419-000.pdf: shape (1, 768)

Total example features loaded: 5


In [11]:
# Test 4: Check similarity between examples
print("Similarity matrix between first 5 examples:")
similarity_matrix = np.zeros((min(5, len(example_features)), min(5, len(example_features))))

for i in range(min(5, len(example_features))):
    for j in range(min(5, len(example_features))):
        sim = cosine_similarity(example_features[i], example_features[j])[0][0]
        similarity_matrix[i, j] = sim

# Display as DataFrame for better readability
sim_df = pd.DataFrame(similarity_matrix, 
                      index=[f"Ex{i}" for i in range(similarity_matrix.shape[0])],
                      columns=[f"Ex{i}" for i in range(similarity_matrix.shape[1])])
print(sim_df.round(3))
print("\nDiagonal should be 1.0 (same image with itself)")

Similarity matrix between first 5 examples:
       Ex0    Ex1    Ex2    Ex3    Ex4
Ex0  1.000  0.827  0.805  0.872  0.770
Ex1  0.827  1.000  0.870  0.841  0.914
Ex2  0.805  0.870  1.000  0.878  0.834
Ex3  0.872  0.841  0.878  1.000  0.799
Ex4  0.770  0.914  0.834  0.799  1.000

Diagonal should be 1.0 (same image with itself)


In [13]:
# Test 5: Replicate the exact detection function
def detect_form_visual_clip_debug(image, clip_model, clip_processor, device,
                                 positive_features=None, negative_features=None,
                                 similarity_threshold=0.7, negative_threshold=0.7):
    """
    Debug version with extra logging
    """
    result = {
        'is_form': False,
        'confidence': 0,
        'max_positive_similarity': 0,
        'max_negative_similarity': 0,
        'positive_similarities': [],
        'negative_similarities': []
    }
    
    # Extract visual features from current image
    try:
        inputs = clip_processor(images=image, return_tensors="pt").to(device)
        
        with torch.no_grad():
            features = clip_model.get_image_features(**inputs)
            features = features.cpu().numpy()
            print(f"  Raw features shape: {features.shape}")
            print(f"  Raw features norm: {np.linalg.norm(features)}")
            
            # Normalize features
            features = features / np.linalg.norm(features, axis=1, keepdims=True)
            print(f"  Normalized features norm: {np.linalg.norm(features)}")
    except Exception as e:
        print(f"Error extracting features: {e}")
        return result
    
    # Check similarity to positive examples
    if positive_features:
        print(f"  Checking against {len(positive_features)} positive examples")
        for i, pos_feat in enumerate(positive_features):
            sim = cosine_similarity(features, pos_feat)[0][0]
            result['positive_similarities'].append(sim)
            print(f"    Example {i}: similarity = {sim:.4f}")
        
        result['max_positive_similarity'] = max(result['positive_similarities'])
        is_like_positive = result['max_positive_similarity'] > similarity_threshold
        print(f"  Max positive similarity: {result['max_positive_similarity']:.4f}")
        print(f"  Is like positive (>{similarity_threshold})? {is_like_positive}")
    else:
        is_like_positive = False
        print("  No positive features provided")
    
    # For now, ignore negative examples to isolate the issue
    is_not_like_negative = True
    
    # Decision: must be like positive AND not like negative
    result['is_form'] = is_like_positive and is_not_like_negative
    print(f"  Final decision: {result['is_form']}")
    
    return result

In [15]:
# Test 6: Test detection on the first example form
print("Testing detection on the first example form:")
print(f"File: {example_files[0]}\n")

# Load the first example
pdf = fitz.open(os.path.join(example_forms_dir, example_files[0]))
page = pdf[0]
pix = page.get_pixmap()
test_img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
pdf.close()

# Run detection
result = detect_form_visual_clip_debug(
    test_img, clip_model, clip_processor, device,
    example_features, None, 0.7, 0.7
)

print(f"\nResult: {result['is_form']}")
print(f"This should be True since we're testing an example against itself!")

Testing detection on the first example form:
File: 25581-000.pdf

  Raw features shape: (1, 768)
  Raw features norm: 16.98063087463379
  Normalized features norm: 0.9999999403953552
  Checking against 5 positive examples
    Example 0: similarity = 1.0000
    Example 1: similarity = 0.8266
    Example 2: similarity = 0.8052
    Example 3: similarity = 0.8720
    Example 4: similarity = 0.7704
  Max positive similarity: 1.0000
  Is like positive (>0.7)? True
  Final decision: True

Result: True
This should be True since we're testing an example against itself!


In [17]:
# Test 7: Check if the issue is with feature dimensions
print("Checking feature dimensions:")
for i, feat in enumerate(example_features[:3]):
    print(f"Example {i}: shape = {feat.shape}, dtype = {feat.dtype}")
    print(f"  Min value: {feat.min():.4f}, Max value: {feat.max():.4f}")
    print(f"  Mean: {feat.mean():.4f}, Std: {feat.std():.4f}")
    print()

Checking feature dimensions:
Example 0: shape = (1, 768), dtype = float32
  Min value: -0.3834, Max value: 0.4124
  Mean: -0.0008, Std: 0.0361

Example 1: shape = (1, 768), dtype = float32
  Min value: -0.3543, Max value: 0.3800
  Mean: -0.0008, Std: 0.0361

Example 2: shape = (1, 768), dtype = float32
  Min value: -0.4028, Max value: 0.4316
  Mean: -0.0009, Std: 0.0361



In [19]:
# Test 8: Try different similarity thresholds
print("Testing different similarity thresholds:\n")

thresholds = [0.5, 0.6, 0.7, 0.8, 0.9]
for threshold in thresholds:
    detected = 0
    for i, example_file in enumerate(example_files[:5]):
        # Load example
        pdf = fitz.open(os.path.join(example_forms_dir, example_file))
        page = pdf[0]
        pix = page.get_pixmap()
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        pdf.close()
        
        # Extract features
        inputs = clip_processor(images=img, return_tensors="pt").to(device)
        with torch.no_grad():
            features = clip_model.get_image_features(**inputs)
            features = features.cpu().numpy()
            features = features / np.linalg.norm(features, axis=1, keepdims=True)
        
        # Check similarity
        max_sim = 0
        for pos_feat in example_features:
            sim = cosine_similarity(features, pos_feat)[0][0]
            max_sim = max(max_sim, sim)
        
        if max_sim > threshold:
            detected += 1
    
    print(f"Threshold {threshold}: {detected}/5 examples detected")

Testing different similarity thresholds:

Threshold 0.5: 5/5 examples detected
Threshold 0.6: 5/5 examples detected
Threshold 0.7: 5/5 examples detected
Threshold 0.8: 5/5 examples detected
Threshold 0.9: 5/5 examples detected
